In [2]:
import training

In [3]:
training.model_summary()

Model: "food_recommender_model_sequence"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
recipe_embedding (RecipeEmbe multiple                  12466112  
_________________________________________________________________
recommender_encoder (Recomme multiple                  330240    
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_5 (Dense)              multiple                  590336    
_________________________________________________________________
dense_6 (Dense)              multiple                  131328    
_________________________________________________________________
dense_7 (Dense)              multiple                  32896     
_________________________________________________________________
dense_8 (Dense)              multip

In [4]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 2464), started 1:46:14 ago. (Use '!kill 2464' to kill it.)

In [5]:
from FoodRatingDataSet import FoodRatingDataset

dataset = FoodRatingDataset()
data = dataset.data()
model = training.create_model(9)
training.train(data, model)

new model:
Epoch 1/10
    240/Unknown - 82s 327ms/step - loss: 12.0769 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
from TestDataSet import TestDataSet

dataset = TestDataSet()
data = dataset.data()
model = training.create_model(5)
training.train(data, model)